In [1]:
import time
import pandas as pd
from datetime import datetime
from appium import webdriver
from appium.options.android import UiAutomator2Options
from appium.webdriver.common.appiumby import AppiumBy
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# ===================== [설정 영역: 잡코리아] =====================
APP_PACKAGE = "com.jobkorea.app"
APP_ACTIVITY = "com.jobkorea.app.view.main.MainAct"
DEVICE_NAME = "Galaxy S25"
APPIUM_SERVER_URL = "http://127.0.0.1:4723"

# 반복 횟수
REPEAT_COUNT = 10

# 검색어 입력값
KEYWORD = "디자이너"

In [3]:
# 메인 테스트

def run_jobkorea_search_test():
    options = UiAutomator2Options()
    options.device_name = DEVICE_NAME
    options.app_package = APP_PACKAGE
    options.app_activity = APP_ACTIVITY
    options.automation_name = "UiAutomator2"
    options.no_reset = True
    options.new_command_timeout = 300
    
    options.set_capability("connectHardwareKeyboard", True)

    print(f"--- 잡코리아(JobKorea) 검색 성능 측정 ({REPEAT_COUNT}회) 시작 ---")
    
    driver = None
    results = []

    try:
        driver = webdriver.Remote(APPIUM_SERVER_URL, options=options)
        wait = WebDriverWait(driver, 20)

        # 화면 크기 계산 (좌표 터치용)
        window_size = driver.get_window_size()
        enter_x = int(window_size['width'] * 0.9)
        enter_y = int(window_size['height'] * 0.9)

        print("📱 앱 실행 및 메인 화면 대기 중...")
        wait.until(EC.presence_of_element_located(
            (AppiumBy.ID, "com.jobkorea.app:id/inputboxLayout")
        ))

        # ===================== 반복 측정 루프 =====================
        for i in range(1, REPEAT_COUNT + 1):
            try:
                print(f"\n[Running] {i}/{REPEAT_COUNT}회차 측정 진행 중...")

                # Step 1. 메인 검색바 클릭
                main_search_bar = wait.until(EC.element_to_be_clickable(
                    (AppiumBy.ID, "com.jobkorea.app:id/inputboxLayout")
                ))
                main_search_bar.click()
                
                # Step 2. 검색어 입력
                time.sleep(1) 
                driver.switch_to.active_element.send_keys(KEYWORD)
                
                # Step 3. 좌표 터치로 검색 실행 (T1)
                print("👆 화면 우측 하단(키보드 엔터) 터치")
                start_time = time.time() 
                driver.tap([(enter_x, enter_y)])
                
                # ---------------------------------------------------------
                # Step 4. 결과 화면 검증 (수정됨: '관련도순' 텍스트 사용)
                # ---------------------------------------------------------
                # '관련도순' 텍스트가 보이면 로딩 끝난 것으로 간주
                wait.until(EC.presence_of_element_located(
                    (AppiumBy.XPATH, "//*[contains(@text, '관련도순')]")
                ))
                
                end_time = time.time()
                
                duration = end_time - start_time
                print(f"⏱️ {i}회차 소요 시간: {duration:.4f}초")

                results.append({
                    "회차": i,
                    "측정시각": datetime.now().strftime("%H:%M:%S"),
                    "검색어": KEYWORD,
                    "검색반응속도(초)": round(duration, 4)
                })

                # Step 5. 메인 화면 복귀
                print("🔙 하드웨어 뒤로가기 키 입력 (2회)")
                driver.press_keycode(4) 
                time.sleep(0.8)
                driver.press_keycode(4)

                wait.until(EC.presence_of_element_located(
                    (AppiumBy.ID, "com.jobkorea.app:id/inputboxLayout")
                ))
                time.sleep(1) 

            except Exception as e:
                print(f"❌ {i}회차 실행 중 에러 발생: {e}")
                try:
                    driver.press_keycode(4)
                    time.sleep(1)
                    driver.press_keycode(4)
                except:
                    pass

        # ===================== CSV 저장 =====================
        print("\n" + "=" * 50)
        print("💾 결과 저장 중...")

        if results:
            df = pd.DataFrame(results)
            speeds = df["검색반응속도(초)"]
            
            summary = {
                "회차": "요약",
                "검색반응속도(초)": "",
                "평균(초)": round(speeds.mean(), 4),
                "최소(초)": round(speeds.min(), 4),
                "최대(초)": round(speeds.max(), 4),
                "표준편차(초)": round(speeds.std(ddof=1), 4) if len(speeds) >= 2 else 0.0
            }
            
            df = pd.concat([df, pd.DataFrame([summary])], ignore_index=True)
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            file_name = f"jobkorea_search_perf_{REPEAT_COUNT}runs_{timestamp}.csv"
            df.to_csv(file_name, index=False, encoding="utf-8-sig")

            print(f"✅ 저장 완료: {file_name}")
            print(df)
        else:
            print("ℹ️ 저장할 데이터가 없습니다.")

    except Exception as e:
        print(f"⛔ 치명적 오류: {e}")

    finally:
        if driver:
            driver.quit()

In [4]:
# 테스트 실행
if __name__ == "__main__":
    run_jobkorea_search_test()

--- 잡코리아(JobKorea) 검색 성능 측정 (10회) 시작 ---
📱 앱 실행 및 메인 화면 대기 중...

[Running] 1/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 1회차 소요 시간: 1.7658초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 2/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 2회차 소요 시간: 1.5580초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 3/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 3회차 소요 시간: 1.3818초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 4/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 4회차 소요 시간: 1.3795초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 5/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 5회차 소요 시간: 1.4491초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 6/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 6회차 소요 시간: 1.4701초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 7/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 7회차 소요 시간: 1.3948초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 8/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 8회차 소요 시간: 1.5004초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 9/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 9회차 소요 시간: 1.4631초
🔙 하드웨어 뒤로가기 키 입력 (2회)

[Running] 10/10회차 측정 진행 중...
👆 화면 우측 하단(키보드 엔터) 터치
⏱️ 10회차 소요 시간: 1.4929초
🔙 하드웨어